## **3. Pondération statistique** (TF-IDF / OKapiBM25)  

https://stackoverflow.com/questions/46580932/calculate-tf-idf-using-sklearn-for-n-grams-in-python  
http://scikit-learn.sourceforge.net/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn-feature-extraction-text-tfidfvectorizer  
https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction

https://pypi.org/project/rank-bm25/

In [58]:
from os import listdir, path
from pandas import *
import os, shutil, re
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import RegexpTokenizer
from rank_bm25 import BM25Okapi
from nltk.tokenize import RegexpTokenizer
regex = "[\w+-]+|\([\s+\w+\d+-]+\)|\w+|\w"
tokenizex = RegexpTokenizer(regex)

corpus_global = []
base_path = '../04-filtrage/output/'
for file in listdir(base_path):
    if file.endswith('.csv'):
        with open(base_path + file, encoding = 'utf-8',) as f:
            corpus_global.append({'acteur': file[:-20], 'N_fr': 0, 'N_MeSH' : 0, 'N_Taxo' : 0}) 
            
N_termes = []
corpus_global

[{'acteur': 'chum', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'chuqc', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'chusj', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'cisss_ca', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'cisss_cotenord', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'cisss_gaspesie', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'cisss_iles', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'cisss_lanaudiere', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'cisss_outaouais', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_at', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_centreouest', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_centresud', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_cn', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_estmtl', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_mcq', 'N_fr': 0, 'N_MeSH': 0, 'N_Taxo': 0},
 {'acteur': 'ciusss_nord

In [59]:
for x in range(len(corpus_global)) :  
    acteur = corpus_global[x]['acteur']

    base_path = '../04-filtrage/output/'

    with open(base_path+acteur+'_candidate-terms.csv', encoding='utf-8') as f:
        csv = read_csv(f).drop(columns = ['Unnamed: 0'])

    corpus_global[x]['N_MeSH'] = len(csv[csv['isMeSHTerm'] == True])
    corpus_global[x]['N_Taxo'] = len(csv[csv['isTaxoTerm'] == True])

    # On va remplacer les apostrophes par des espaces ; sklearn ne les aime pas 
    csv['Terme formatté'] = csv["Terme"].apply(lambda x: x.replace("'", ' '))
    vocabulaire = [t.lower() for t in list(csv['Terme formatté'])]
    print(acteur, 'On a un vocabulaire de {} formes.'.format(len(vocabulaire)))
    N_termes += vocabulaire

    # Lire le corpus
    base_path = '../03-corpus/2-data/1-fr/'
    file_path = path.join(base_path, acteur + '.csv')
        
    with open(file_path, "r", encoding = "UTF-8") as f:
        data = read_csv(file_path, sep=',')
        text = data['text'].dropna().tolist()
        nb_docs = len(text)

    corpus_global[x]['N_fr'] = nb_docs
    print("On a un corpus de {} documents.".format(nb_docs))

    # Nettoyage
    corpus = [str(t).strip('\n').lower().replace('’', '\'') for t in text]
    spaces = '\s+'

    corpus = [re.sub(spaces, ' ', t) for t in corpus]
    corpus = [t.replace("  ", " " ) for t in corpus]



    #On va commencer par utiliser le CountVectorizer pour valider que l'implémentation de sklearn arrive bien au même compte que nous 
    vectorizer = CountVectorizer(tokenizer = tokenizex.tokenize, vocabulary=vocabulaire, ngram_range=(1,10), token_pattern=regex)
    X = vectorizer.fit_transform(corpus)
    vectorizer.get_feature_names_out()

    features_names = vectorizer.get_feature_names_out()
    corpus_index = [corpus.index(n) for n in corpus]


    df = DataFrame(X.T.todense(), index=features_names, columns=corpus_index).transpose()

    csv['TF (sklearn)'] = 0
    for t in vocabulaire:
        csv.loc[csv['Terme formatté'] == t, 'TF (sklearn)'] = df[t].sum()


    csv['DF (sklearn)'] = 0
    for t in vocabulaire:
        freqdoc = len(df[df[t] != 0])
        csv.loc[csv['Terme formatté'] == t, 'DF (sklearn)'] = freqdoc
    csv

    # Calcul TF-IDF
    # vocabulary = vocabulaire
    tfidf_vectorizer = TfidfVectorizer(tokenizer = tokenizex.tokenize, vocabulary=vocabulaire, ngram_range=(1,10), lowercase=False, token_pattern=regex)
    tfidf = tfidf_vectorizer.fit_transform(corpus)

    features_names = tfidf_vectorizer.get_feature_names_out()
    corpus_index = [corpus.index(n) for n in corpus]

    df = DataFrame(tfidf.T.todense(), index=features_names, columns=corpus_index).transpose()
    terms_tfidf = {term: df[term].max() for term in df}
    zeros = {term : value for term, value in terms_tfidf.items() if terms_tfidf[term] == 0}
    print('TF-IDF = 0',zeros)
    print('% TF-IDF = 0', (len(zeros) / len(vocabulaire) *100))
    
    terms_weighted = DataFrame(terms_tfidf.items(), columns=['Terme formatté', 'TF-IDF'])
    terms_weighted.sort_values(["TF-IDF"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

    terms_weighted = terms_weighted.drop_duplicates(keep='first')

    terms_weighted = merge(csv, terms_weighted, on='Terme formatté').drop_duplicates(
    subset = ['Terme', 'Fréquence (TF)'],
    keep = 'first').reset_index(drop = True)


    # Calcul OKapi BM25
    tokenizer = RegexpTokenizer("(\w+\'|\w+-\w+|\(|\)|\w+)")
    bm25 = BM25Okapi([tokenizer.tokenize(t) for t in corpus])

    vocabulaire = csv["Terme"].dropna().tolist()
    tokenized_queries = [tokenizer.tokenize(str(t)) for t in vocabulaire]

    features_names = [t for t in set(vocabulaire)]
    corpus_index = [corpus.index(n) for n in corpus]

    tab = [bm25.get_scores(query) for query in tokenized_queries]
    df = DataFrame(tab, index=features_names, columns=corpus_index).transpose()

    terms_okapi = {term: df[term].max() for term in df}
    tab = DataFrame(terms_okapi.items(), columns=['Terme', 'OkapiBM25'])
    tab.sort_values(["OkapiBM25"], 
                    axis=0,
                    ascending=[False], 
                    inplace=True)

    tab = merge(terms_weighted, tab, on="Terme")
    tab.sort_values(["OkapiBM25"], 
                        axis=0,
                        ascending=[False], 
                        inplace=True)

    base_path = '../05-transformation/'
    file_path = base_path + acteur  + '_weighting_OKapiBM25.csv'
    tab.to_csv(file_path)

chum On a un vocabulaire de 459 formes.
On a un corpus de 3562 documents.
TF-IDF = 0 {'centre des rendez': 0.0, 'consultations provenant du chum': 0.0}
% TF-IDF = 0 0.4357298474945534
chuqc On a un vocabulaire de 1407 formes.
On a un corpus de 755 documents.
TF-IDF = 0 {'annulation de rendez': 0.0, 'demande de rendez': 0.0, 'version mots': 0.0, 'rabais offerts': 0.0, 'ancienneté transférable': 0.0, 'prochain rendez': 0.0, 'mots et gestes': 0.0, 'centre de rendez': 0.0, 'mois et version': 0.0, 'mois et version mots': 0.0, 'mots et énoncés': 0.0, 'fiches de la banque provinciale de données': 0.0, 'version mots et gestes': 0.0, 'fiches de la banque provinciale': 0.0, 'objectifs ciblés': 0.0, 'fiches de la banque': 0.0, 'cotation des fiches de la banque': 0.0, 'cotation des fiches': 0.0, 'cotation des fiches de la banque provinciale': 0.0, 'formation préalable': 0.0, 'formation préalable obligatoire': 0.0, 'mikroport 2015. instructions pour l utilisation': 0.0, 'système mf mikroport 2015. 

In [ ]:
N_termes = set(N_termes)

In [ ]:
N_termes

{'évaluation addnutriv agent de suppléance',
 'accès au portail extranet',
 'infirmière pivot au centre',
 'gestion des maladies chroniques',
 'cours des prochaines semaines',
 'enjeux éthiques',
 'problèmes reliés',
 'récoltes fertiles',
 'québec _ramq_',
 'attente d_évaluation tavneos',
 'gestion des transferts',
 'vélo sécurisé pour les patients et les visiteurs',
 'grâce au service carnet de santé',
 'cas d_urgence',
 'renseignements décrivant',
 'chaleur accablante contamination des aliments contamination',
 'reconnaissance portraits',
 'québec procédure_de demande_d_exemptions_par_le_medecin_ou_l_ips vaccination',
 'recherche éthique',
 'application des mesures',
 'soins accès',
 'thérapeute en réadaptation',
 'échelle edss',
 'situation de maltraitance',
 'évaluation radicava ors édaravone',
 'service de buanderie',
 'asthme sévère de type',
 'exigences pré_analytiques',
 'action bénévole',
 'cours de réalisation par l_inesss',
 'services de garde',
 'infections _pci_',
 'azithr

In [ ]:
len(N_termes)

25721

In [ ]:
corpus_global

[{'acteur': 'chum', 'N_fr': 3568, 'N_MeSH': 40, 'N_Taxo': 28},
 {'acteur': 'chuqc', 'N_fr': 756, 'N_MeSH': 71, 'N_Taxo': 54},
 {'acteur': 'chusj', 'N_fr': 786, 'N_MeSH': 38, 'N_Taxo': 27},
 {'acteur': 'cisss_ca', 'N_fr': 3306, 'N_MeSH': 31, 'N_Taxo': 22},
 {'acteur': 'cisss_cotenord', 'N_fr': 520, 'N_MeSH': 33, 'N_Taxo': 31},
 {'acteur': 'cisss_gaspesie', 'N_fr': 866, 'N_MeSH': 47, 'N_Taxo': 36},
 {'acteur': 'cisss_iles', 'N_fr': 111, 'N_MeSH': 16, 'N_Taxo': 12},
 {'acteur': 'cisss_lanaudiere', 'N_fr': 678, 'N_MeSH': 68, 'N_Taxo': 48},
 {'acteur': 'cisss_outaouais', 'N_fr': 1163, 'N_MeSH': 48, 'N_Taxo': 44},
 {'acteur': 'ciusss_at', 'N_fr': 266, 'N_MeSH': 38, 'N_Taxo': 31},
 {'acteur': 'ciusss_centreouest', 'N_fr': 730, 'N_MeSH': 58, 'N_Taxo': 44},
 {'acteur': 'ciusss_centresud', 'N_fr': 802, 'N_MeSH': 48, 'N_Taxo': 46},
 {'acteur': 'ciusss_cn', 'N_fr': 3597, 'N_MeSH': 113, 'N_Taxo': 70},
 {'acteur': 'ciusss_estmtl', 'N_fr': 1127, 'N_MeSH': 34, 'N_Taxo': 35},
 {'acteur': 'ciusss_mcq', 